In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import * 

In [2]:
X_train = [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_train = [1, 0, 1, 0]

X_test =  [[-1, -1],  [-1,1], [1,1], [1,-1]]
y_test =  [1, 0, 1, 0]

In [3]:
with open('results/version7/train_xor.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version7/test_xor.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)

# Search

In [4]:
#inputVector = [math.atan(inputVector[0]), math.atan(inputVector[1])]
#inputVector = [math.sqrt(inputVector[0]**2), math.sqrt(inputVector[1]**2)]

In [5]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = [0.5, 0.5],
                           threshold=0.3,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=True,
                           #epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs','phase-encoding'})

lrParameter:  0.1
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  2
best error HSGS training:  2


In [6]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.3,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=True, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs', 'phase-encoding'})

AVG TEST ERROR HSGS    0.5
AVG TEST ERROR PHASE   0.5


In [7]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

print('phase neuron_outputs: ', results[1])
print('HSGS neuron_outputs: ', results[4])
print('targets: ', results[6])
print('examples: ', results[7])

phase avg_error 0.5
phase weights [0.5, 0.5, 4.999999999999999, 4.999999999999999]
\HSGS avg_error 0.5
HSGS weights [0.5, 0.5, 4.999999999999999, 4.999999999999999]
phase neuron_outputs:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
HSGS neuron_outputs:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
targets:  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
examples:  [[-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1], [-1, -1], [-1, 1], [1, 1], [1, -1]]


In [4]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [5]:
#[list(np.random.rand(2)) for i in range(5)]

In [6]:
# search space

initial_weights = [
 [0.5, 0.5],
 [0.9091283748867313, 0.13316944575925016],
 #[0.5234125806737658, 0.7504098591020348],
 [0.6690132408839138, 0.4677528597449807],
 #[0.20484909029779508, 0.49076588909107044],
 #[0.372384689385059, 0.4774011548515884]
]

threshold_space = [0.1, 0.3, 0.5, 0.7, 0.8, 0.9]
lr_space = [0.01, 0.02, 0.05, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [7]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=w,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs','phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=10,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'hsgs', 'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)

                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')


lrParameter:  0.01
threshold:  0.1
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2
best error HSGS training:  2
AVG TEST ERROR HSGS    0.5
AVG TEST ERROR PHASE   0.5


lrParameter:  0.01
threshold:  0.1
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  2
best error HSGS training:  2
AVG TEST ERROR HSGS    0.5
AVG TEST ERROR PHASE   0.5


lrParameter:  0.02
threshold:  0.1
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2
best error HSGS training:  2
AVG TEST ERROR HSGS    0.5
AVG TEST ERROR PHASE   0.5


lrParameter:  0.02
threshold:  0.1
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  2
best error HSGS training:  2
AVG TEST ERROR HSGS    0.5
AVG TEST ERROR PHASE   0.5


lrParameter:  0.05
threshold:  0.1
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  2
best error H

KeyboardInterrupt: 

In [10]:
experiment_data = pd.DataFrame(output)
experiment_data

,model,phase_strategy,bias,threshold,lr,avg_error,trained_weights,initial_weights,neuron_outputs
0,phase-encoding,original,False,0.1,0.01,0.5,"[0.5, 0.5]","[0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,HSGS,original,False,0.1,0.01,0.5,"[0.5, 0.5]","[0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,phase-encoding,original,True,0.1,0.01,0.5,"[0.5, 0.5, 1.4000000000000004, 1.4000000000000...","[0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,HSGS,original,True,0.1,0.01,0.5,"[0.5, 0.5, 1.4000000000000004, 1.4000000000000...","[0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,phase-encoding,original,False,0.1,0.02,0.5,"[0.5, 0.5]","[0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...,...,...
991,HSGS,radius,True,0.3,0.10,0.5,"[0.6690132408839138, 0.4677528597449807, 4.999...","[0.6690132408839138, 0.4677528597449807]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
992,phase-encoding,radius,False,0.5,0.01,0.5,"[0.3861705284092953, 0.4677528597449807]","[0.6690132408839138, 0.4677528597449807]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
993,HSGS,radius,False,0.5,0.01,0.5,"[0.6690132408839138, 0.4677528597449807]","[0.6690132408839138, 0.4677528597449807]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
994,phase-encoding,radius,True,0.5,0.01,0.5,"[1.2346986658331516, 1.0334382846942172, 1.0, ...","[0.6690132408839138, 0.4677528597449807]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [11]:
experiment_data.to_csv('experiment_xor_lite.csv')

- adaptar angle e radius para > 2
- teste com base xor real
- teste com base não linear real